In [1]:
import sys
sys.path.append('../../../')

In [2]:
from copy import deepcopy
import src.requestcompletion as rc
from tests.rc_tests.fixtures.nodes import ActionType, CapitalizeText, FatalErrorNode, UnknownErrorNode, TimeoutNode
import warnings

In [8]:
model = rc.llm.OpenAILLM("gpt-4o")

system_rng = rc.llm.SystemMessage("You are a random integer generator that will return a random list of integers between 0 and 100. Do not return more than 10 integers.")
system_rng_operation = rc.llm.SystemMessage("You are a random mathematical operation calculator that will apply a random operation to the list of integers that will be provided by the user and return the result. The answer should be only a single integer.")
system_math_genius = rc.llm.SystemMessage("You are a math genius that will get a list of integers(loi) and another interger(x), your task is to predict what operation must be appled to the list of integers to get the result of x.")

RNGNode = rc.library.terminal_llm("COT", system_message=system_rng, model=model)
RNGOperationNode = rc.library.terminal_llm("Answerer", system_message=system_rng_operation, model=model)
class MathDetectiveNode(rc.library.TerminalLLM):
    def __init__(self, message_history: rc.llm.MessageHistory, llm_model: rc.llm.ModelBase, system_message: rc.llm.SystemMessage = system_math_genius):
        if len([x for x in message_history if x.role == "system"]) > 0:
            warnings.warn("System message already exists in message history. We will replace it.")
            message_history = [x for x in message_history if x.role != "system"]
            message_history.insert(0, system_message)
        else:
            message_history.insert(0, system_message)
        
        super().__init__(message_history=message_history, model=llm_model)
    
    def pretty_name(self) -> str:
        return "Math Detective Node"


async def MathGame(message_history: rc.llm.MessageHistory):
    
    # Use the working copy for all subsequent calls
    random_num_list_response = await rc.call(RNGNode, message_history=message_history)
    message_history.append(rc.llm.AssistantMessage("The list of random integer: " + str(random_num_list_response)))

    operation_response = await rc.call(RNGOperationNode, message_history=message_history)
    message_history.append(rc.llm.AssistantMessage("The result int (x) = " + str(operation_response)))


    response = await rc.call(MathDetectiveNode, message_history=message_history, llm_model=model)

    return response, message_history

    
MathGameNode = rc.library.from_function(MathGame)   # returns a node that can be run


In [9]:
original_message_history = rc.llm.MessageHistory([rc.llm.UserMessage("Start the game")])
original_message_history

[user: Start the game]

In [10]:
# Run the call and store the result
with rc.Runner() as runner:
    result, hist = await runner.run(MathGameNode, message_history=original_message_history)
result.answer

1: 1906914235616
1: 1906920515568
1: 1906920513808


TypeError: cannot unpack non-iterable ExecutionInfo object

In [6]:
print(original_message_history)

user: Start the game
